# Bibliotecas

In [47]:
import random
import copy

# Leitura dos dados

In [48]:
def read_instances(file_path):
    instances = []
    with open(file_path, 'r') as file:
        lines = file.readlines()
        i = 0
        while i < len(lines):
            line = lines[i].strip()
            if line.isdigit():
                size = int(line)
                i += 1
                while i < len(lines):
                    if lines[i].strip() == '':
                        i += 1
                        if i < len(lines) and lines[i].strip() == '':
                            i += 1
                            break  # Duas linhas vazias consecutivas, mudar o tamanho da instância
                        continue  # Linha vazia, continuar para a próxima linha
                    numbers = []
                    for _ in range(size):
                        if i < len(lines):
                            num_line = lines[i].strip()
                            if num_line:
                                numbers.append(int(num_line))
                            i += 1
                    instances.append(numbers)
            else:
                i += 1  # Pula linhas vazias ou não numéricas
    return instances

# Exemplo de uso
file_path = 'dados_literatura_Particao_Numeros.txt'  # Ajustar o caminho conforme necessário
instances = read_instances(file_path)

# Verificando a leitura das instâncias
for idx, instance in enumerate(instances):
    print(f"Instância {idx+1} ({len(instance)} números): {instance}")

Instância 1 (15 números): [1860593326, 5956064568, 8784577856, 4894035213, 716642272, 9233584296, 2338836548, 1220884556, 6222901391, 9764627608, 109435896, 854909134, 477886997, 1743072596, 3641281477]
Instância 2 (15 números): [5128991022, 4532281013, 2537593172, 771452685, 5415152717, 4872884847, 4930454686, 9046553222, 5129738035, 1440703686, 8548266266, 9100734872, 9441006496, 491294914, 9256635732]
Instância 3 (15 números): [9496327829, 3008335560, 3182609501, 8459047958, 3244364330, 9321283309, 7532073923, 6891220098, 4849464750, 1393776662, 7994090908, 8386779510, 9206324775, 7251315129, 3950961279]
Instância 4 (15 números): [1854855597, 3785471115, 5966617027, 4146562220, 1063583983, 6769583879, 9132701079, 4716977964, 4666090587, 4076652849, 8441094368, 7454624448, 8279474265, 4189737317, 875115524]
Instância 5 (15 números): [5132461393, 2241395660, 6711623464, 2825959894, 6773093408, 2216884340, 2734329117, 532544630, 1386706201, 4942648847, 6089827028, 3820669186, 824579466

# ILS_PR

In [49]:
import random

def generate_initial_solution(numbers, greedy=True):
    # Implementação de uma solução inicial (pode ser gulosa ou aleatória)
    return [random.randint(0, 1) for _ in range(len(numbers))]

def evaluate_solution(solution, numbers):
    subset1 = sum(numbers[i] for i in range(len(numbers)) if solution[i] == 0)
    subset2 = sum(numbers[i] for i in range(len(numbers)) if solution[i] == 1)
    return abs(subset1 - subset2)

def local_search(solution, numbers):
    # Implementação de uma busca local simples
    best_solution = solution[:]
    best_cost = evaluate_solution(solution, numbers)
    for i in range(len(solution)):
        new_solution = solution[:]
        new_solution[i] = 1 - new_solution[i]  # Troca 0 por 1 ou 1 por 0
        new_cost = evaluate_solution(new_solution, numbers)
        if new_cost < best_cost:
            best_solution = new_solution
            best_cost = new_cost
    return best_solution

def perturb(solution):
    # Implementação de uma perturbação simples
    new_solution = solution[:]
    for _ in range(len(solution) // 10):  # Perturba 10% dos elementos
        index = random.randint(0, len(solution) - 1)
        new_solution[index] = 1 - new_solution[index]
    return new_solution

def path_relinking(solution1, solution2, numbers):
    # Implementação de Path Relinking
    best_solution = solution1[:]
    best_cost = evaluate_solution(solution1, numbers)
    for i in range(len(solution1)):
        if solution1[i] != solution2[i]:
            new_solution = solution1[:]
            new_solution[i] = solution2[i]
            new_cost = evaluate_solution(new_solution, numbers)
            if new_cost < best_cost:
                best_solution = new_solution
                best_cost = new_cost
    return best_solution

def ils_with_path_relinking(numbers, max_iterations=100, elite_size=5):
    # Gerar solução inicial
    current_solution = generate_initial_solution(numbers, greedy=True)
    current_cost = evaluate_solution(current_solution, numbers)
    
    # Grupo elite de soluções
    elite_group = [current_solution]
    
    for iteration in range(max_iterations):
        # Busca Local
        current_solution = local_search(current_solution, numbers)
        current_cost = evaluate_solution(current_solution, numbers)
        
        # Verifica se a solução entra no grupo elite
        if not elite_group or current_cost < evaluate_solution(elite_group[-1], numbers):
            elite_group.append(current_solution)
            elite_group = sorted(elite_group, key=lambda s: evaluate_solution(s, numbers))[:elite_size]  # Mantém grupo elite
        
        # Perturbação
        perturbed_solution = perturb(current_solution)
        
        # Busca Local na solução perturbada
        perturbed_solution = local_search(perturbed_solution, numbers)
        
        # Path Relinking
        for elite_solution in elite_group:
            perturbed_solution = path_relinking(perturbed_solution, elite_solution, numbers)
        
        # Critério de aceitação
        if evaluate_solution(perturbed_solution, numbers) < current_cost:
            current_solution = perturbed_solution
        
    # Retorna a melhor solução encontrada
    best_solution = min(elite_group, key=lambda s: evaluate_solution(s, numbers))
    return best_solution, evaluate_solution(best_solution, numbers)

# Exemplo de uso com leitura do arquivo
file_path = 'dados_literatura_Particao_Numeros.txt'  # Ajustar o caminho conforme necessário
instances = read_instances(file_path)

# Aplicando o algoritmo de ILS com Path Relinking em cada instância
for idx, numbers in enumerate(instances):
    print(f"Instância {idx+1} ({len(numbers)} números):")
    best_solution, best_cost = ils_with_path_relinking(numbers)
    print(f"Melhor solução: {best_solution}")
    print(f"Diferença mínima entre os subconjuntos: {best_cost}\n")

Instância 1 (15 números):
Melhor solução: [1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1]
Diferença mínima entre os subconjuntos: 98801424

Instância 2 (15 números):
Melhor solução: [1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1]
Diferença mínima entre os subconjuntos: 31948917

Instância 3 (15 números):
Melhor solução: [1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1]
Diferença mínima entre os subconjuntos: 460928765

Instância 4 (15 números):
Melhor solução: [1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1]
Diferença mínima entre os subconjuntos: 588725922

Instância 5 (15 números):
Melhor solução: [1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0]
Diferença mínima entre os subconjuntos: 104740360

Instância 6 (35 números):
Melhor solução: [0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0]
Diferença mínima entre os subconjuntos: 20840002

Instância 7 (35 números):
Melhor solução: [0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 

In [50]:
import random
import math

def simulated_annealing(numbers, initial_temp, cooling_rate, min_temp):
    def evaluate_solution(solution, numbers):
        subset1 = sum(numbers[i] for i in range(len(numbers)) if solution[i] == 0)
        subset2 = sum(numbers[i] for i in range(len(numbers)) if solution[i] == 1)
        return abs(subset1 - subset2)
    
    def perturb(solution):
        new_solution = solution[:]
        index = random.randint(0, len(solution) - 1)
        new_solution[index] = 1 - new_solution[index]  # Troca 0 por 1 ou 1 por 0
        return new_solution
    
    # Inicializar solução
    current_solution = [random.randint(0, 1) for _ in range(len(numbers))]
    current_cost = evaluate_solution(current_solution, numbers)
    best_solution = current_solution[:]
    best_cost = current_cost
    
    temperature = initial_temp
    
    while temperature > min_temp:
        new_solution = perturb(current_solution)
        new_cost = evaluate_solution(new_solution, numbers)
        
        if new_cost < current_cost or random.random() < math.exp((current_cost - new_cost) / temperature):
            current_solution = new_solution
            current_cost = new_cost
            
            if current_cost < best_cost:
                best_solution = current_solution
                best_cost = current_cost
        
        temperature *= cooling_rate
    
    return best_solution, best_cost

# Exemplo de uso com leitura do arquivo
file_path = 'dados_literatura_Particao_Numeros.txt'  # Ajustar o caminho conforme necessário
instances = read_instances(file_path)

# Aplicando o algoritmo de Simulated Annealing em cada instância
initial_temp = 1000
cooling_rate = 0.95
min_temp = 1

for idx, numbers in enumerate(instances):
    print(f"Instância {idx+1} ({len(numbers)} números):")
    best_solution, best_cost = simulated_annealing(numbers, initial_temp, cooling_rate, min_temp)
    print(f"Melhor solução: {best_solution}")
    print(f"Diferença mínima entre os subconjuntos: {best_cost}\n")

Instância 1 (15 números):
Melhor solução: [1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0]
Diferença mínima entre os subconjuntos: 51585072

Instância 2 (15 números):
Melhor solução: [1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0]
Diferença mínima entre os subconjuntos: 171440475

Instância 3 (15 números):
Melhor solução: [0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0]
Diferença mínima entre os subconjuntos: 10410107

Instância 4 (15 números):
Melhor solução: [0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0]
Diferença mínima entre os subconjuntos: 319237610

Instância 5 (15 números):
Melhor solução: [0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1]
Diferença mínima entre os subconjuntos: 1192634828

Instância 6 (35 números):
Melhor solução: [1, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0]
Diferença mínima entre os subconjuntos: 285186014

Instância 7 (35 números):
Melhor solução: [1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1

# Algoritmo de Karmarkar-Karp (KK)

In [51]:
import heapq

def karmarkar_karp(numbers):
    # Usar uma fila de prioridade (heap) para manter os números em ordem decrescente
    heap = [-num for num in numbers]
    heapq.heapify(heap)
    
    while len(heap) > 1:
        largest = -heapq.heappop(heap)
        second_largest = -heapq.heappop(heap)
        difference = largest - second_largest
        heapq.heappush(heap, -difference)
    
    return -heap[0]

# Exemplo de uso com leitura do arquivo
file_path = 'dados_literatura_Particao_Numeros.txt'  # Ajustar o caminho conforme necessário
instances = read_instances(file_path)

# Aplicando o algoritmo de Karmarkar-Karp em cada instância
for idx, numbers in enumerate(instances):
    print(f"Instância {idx+1} ({len(numbers)} números):")
    min_difference = karmarkar_karp(numbers)
    print(f"Diferença mínima entre os subconjuntos: {min_difference}\n")

Instância 1 (15 números):
Diferença mínima entre os subconjuntos: 2043812

Instância 2 (15 números):
Diferença mínima entre os subconjuntos: 49752055

Instância 3 (15 números):
Diferença mínima entre os subconjuntos: 14440579

Instância 4 (15 números):
Diferença mínima entre os subconjuntos: 244401558

Instância 5 (15 números):
Diferença mínima entre os subconjuntos: 11307616

Instância 6 (35 números):
Diferença mínima entre os subconjuntos: 24124

Instância 7 (35 números):
Diferença mínima entre os subconjuntos: 537372

Instância 8 (35 números):
Diferença mínima entre os subconjuntos: 195668

Instância 9 (35 números):
Diferença mínima entre os subconjuntos: 16408

Instância 10 (35 números):
Diferença mínima entre os subconjuntos: 719839

Instância 11 (55 números):
Diferença mínima entre os subconjuntos: 135442

Instância 12 (55 números):
Diferença mínima entre os subconjuntos: 67872

Instância 13 (55 números):
Diferença mínima entre os subconjuntos: 21302

Instância 14 (55 números):
D

# GRASP com Path Relinking

In [52]:
import random

def grasp_with_path_relinking(numbers, max_iterations=100, rcl_size=5, elite_size=5):
    def evaluate_solution(solution, numbers):
        subset1 = sum(numbers[i] for i in range(len(numbers)) if solution[i] == 0)
        subset2 = sum(numbers[i] for i in range(len(numbers)) if solution[i] == 1)
        return abs(subset1 - subset2)
    
    def greedy_randomized_construction(numbers, rcl_size):
        solution = [0] * len(numbers)
        remaining_indices = list(range(len(numbers)))
        while remaining_indices:
            rcl = random.sample(remaining_indices, min(rcl_size, len(remaining_indices)))
            selected_index = random.choice(rcl)
            solution[selected_index] = 1 if sum(numbers[i] for i in range(len(numbers)) if solution[i] == 0) > sum(numbers[i] for i in range(len(numbers)) if solution[i] == 1) else 0
            remaining_indices.remove(selected_index)
        return solution
    
    def local_search(solution, numbers):
        best_solution = solution[:]
        best_cost = evaluate_solution(solution, numbers)
        for i in range(len(solution)):
            new_solution = solution[:]
            new_solution[i] = 1 - new_solution[i]  # Troca 0 por 1 ou 1 por 0
            new_cost = evaluate_solution(new_solution, numbers)
            if new_cost < best_cost:
                best_solution = new_solution
                best_cost = new_cost
        return best_solution
    
    def path_relinking(solution1, solution2, numbers):
        best_solution = solution1[:]
        best_cost = evaluate_solution(solution1, numbers)
        for i in range(len(solution1)):
            if solution1[i] != solution2[i]:
                new_solution = solution1[:]
                new_solution[i] = solution2[i]
                new_cost = evaluate_solution(new_solution, numbers)
                if new_cost < best_cost:
                    best_solution = new_solution
                    best_cost = new_cost
        return best_solution
    
    best_solution = None
    best_cost = float('inf')
    elite_solutions = []
    
    for _ in range(max_iterations):
        solution = greedy_randomized_construction(numbers, rcl_size)
        solution = local_search(solution, numbers)
        cost = evaluate_solution(solution, numbers)
        
        if len(elite_solutions) < elite_size:
            elite_solutions.append(solution)
        else:
            worst_elite = max(elite_solutions, key=lambda s: evaluate_solution(s, numbers))
            if cost < evaluate_solution(worst_elite, numbers):
                elite_solutions.remove(worst_elite)
                elite_solutions.append(solution)
        
        for elite_solution in elite_solutions:
            solution = path_relinking(solution, elite_solution, numbers)
            cost = evaluate_solution(solution, numbers)
            if cost < best_cost:
                best_solution = solution
                best_cost = cost
    
    return best_solution, best_cost

# Exemplo de uso com leitura do arquivo
file_path = 'dados_literatura_Particao_Numeros.txt'  # Ajustar o caminho conforme necessário
instances = read_instances(file_path)

# Aplicando o algoritmo de GRASP com Path Relinking em cada instância
for idx, numbers in enumerate(instances):
    print(f"Instância {idx+1} ({len(numbers)} números):")
    best_solution, best_cost = grasp_with_path_relinking(numbers)
    print(f"Melhor solução: {best_solution}")
    print(f"Diferença mínima entre os subconjuntos: {best_cost}\n")

Instância 1 (15 números):
Melhor solução: [1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1]
Diferença mínima entre os subconjuntos: 1257426

Instância 2 (15 números):
Melhor solução: [1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0]
Diferença mínima entre os subconjuntos: 3142889

Instância 3 (15 números):
Melhor solução: [0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0]
Diferença mínima entre os subconjuntos: 4921863

Instância 4 (15 números):
Melhor solução: [0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0]
Diferença mínima entre os subconjuntos: 27177188

Instância 5 (15 números):
Melhor solução: [0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1]
Diferença mínima entre os subconjuntos: 36172

Instância 6 (35 números):
Melhor solução: [1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1]
Diferença mínima entre os subconjuntos: 4423648

Instância 7 (35 números):
Melhor solução: [0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0

# GRASP com Simulated Annealing


In [53]:
import random
import math

def grasp_with_simulated_annealing(numbers, max_iterations=100, rcl_size=5, initial_temp=1000, cooling_rate=0.95, min_temp=1):
    def evaluate_solution(solution, numbers):
        subset1 = sum(numbers[i] for i in range(len(numbers)) if solution[i] == 0)
        subset2 = sum(numbers[i] for i in range(len(numbers)) if solution[i] == 1)
        return abs(subset1 - subset2)
    
    def greedy_randomized_construction(numbers, rcl_size):
        solution = [0] * len(numbers)
        remaining_indices = list(range(len(numbers)))
        while remaining_indices:
            rcl = random.sample(remaining_indices, min(rcl_size, len(remaining_indices)))
            selected_index = random.choice(rcl)
            solution[selected_index] = 1 if sum(numbers[i] for i in range(len(numbers)) if solution[i] == 0) > sum(numbers[i] for i in range(len(numbers)) if solution[i] == 1) else 0
            remaining_indices.remove(selected_index)
        return solution
    
    def simulated_annealing(solution, numbers, initial_temp, cooling_rate, min_temp):
        current_solution = solution[:]
        current_cost = evaluate_solution(current_solution, numbers)
        best_solution = current_solution[:]
        best_cost = current_cost
        temperature = initial_temp
        
        while temperature > min_temp:
            new_solution = current_solution[:]
            index = random.randint(0, len(new_solution) - 1)
            new_solution[index] = 1 - new_solution[index]  # Troca 0 por 1 ou 1 por 0
            new_cost = evaluate_solution(new_solution, numbers)
            
            if new_cost < current_cost or random.random() < math.exp((current_cost - new_cost) / temperature):
                current_solution = new_solution
                current_cost = new_cost
                
                if current_cost < best_cost:
                    best_solution = current_solution
                    best_cost = current_cost
            
            temperature *= cooling_rate
        
        return best_solution
    
    best_solution = None
    best_cost = float('inf')
    
    for _ in range(max_iterations):
        solution = greedy_randomized_construction(numbers, rcl_size)
        solution = simulated_annealing(solution, numbers, initial_temp, cooling_rate, min_temp)
        cost = evaluate_solution(solution, numbers)
        if cost < best_cost:
            best_solution = solution
            best_cost = cost
    
    return best_solution, best_cost

# Exemplo de uso com leitura do arquivo
file_path = 'dados_literatura_Particao_Numeros.txt'  # Ajustar o caminho conforme necessário
instances = read_instances(file_path)

# Aplicando o algoritmo de GRASP com Simulated Annealing em cada instância
for idx, numbers in enumerate(instances):
    print(f"Instância {idx+1} ({len(numbers)} números):")
    best_solution, best_cost = grasp_with_simulated_annealing(numbers)
    print(f"Melhor solução: {best_solution}")
    print(f"Diferença mínima entre os subconjuntos: {best_cost}\n")

Instância 1 (15 números):
Melhor solução: [1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1]
Diferença mínima entre os subconjuntos: 3584688

Instância 2 (15 números):
Melhor solução: [1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0]
Diferença mínima entre os subconjuntos: 16113287

Instância 3 (15 números):
Melhor solução: [0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1]
Diferença mínima entre os subconjuntos: 2935219

Instância 4 (15 números):
Melhor solução: [0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1]
Diferença mínima entre os subconjuntos: 2019624

Instância 5 (15 números):
Melhor solução: [0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1]
Diferença mínima entre os subconjuntos: 11307616

Instância 6 (35 números):
Melhor solução: [1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0]
Diferença mínima entre os subconjuntos: 1719578

Instância 7 (35 números):
Melhor solução: [1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0

# ILS com Simulated Annealing

In [57]:
import random
import math

def ils_with_simulated_annealing(numbers, max_iterations=100, initial_temp=1000, cooling_rate=0.95, min_temp=1):
    def evaluate_solution(solution, numbers):
        subset1 = sum(numbers[i] for i in range(len(numbers)) if solution[i] == 0)
        subset2 = sum(numbers[i] for i in range(len(numbers)) if solution[i] == 1)
        return abs(subset1 - subset2)
    
    def generate_initial_solution(numbers):
        return [random.randint(0, 1) for _ in range(len(numbers))]
    
    def local_search(solution, numbers):
        best_solution = solution[:]
        best_cost = evaluate_solution(solution, numbers)
        for i in range(len(solution)):
            new_solution = solution[:]
            new_solution[i] = 1 - new_solution[i]  # Troca 0 por 1 ou 1 por 0
            new_cost = evaluate_solution(new_solution, numbers)
            if new_cost < best_cost:
                best_solution = new_solution
                best_cost = new_cost
        return best_solution
    
    def perturb(solution):
        new_solution = solution[:]
        for _ in range(len(solution) // 10):  # Perturba 10% dos elementos
            index = random.randint(0, len(solution) - 1)
            new_solution[index] = 1 - new_solution[index]
        return new_solution
    
    def simulated_annealing(solution, numbers, initial_temp, cooling_rate, min_temp):
        current_solution = solution[:]
        current_cost = evaluate_solution(current_solution, numbers)
        best_solution = current_solution[:]
        best_cost = current_cost
        temperature = initial_temp
        
        while temperature > min_temp:
            new_solution = current_solution[:]
            index = random.randint(0, len(new_solution) - 1)
            new_solution[index] = 1 - new_solution[index]  # Troca 0 por 1 ou 1 por 0
            new_cost = evaluate_solution(new_solution, numbers)
            
            if new_cost < current_cost or random.random() < math.exp((current_cost - new_cost) / temperature):
                current_solution = new_solution
                current_cost = new_cost
                
                if current_cost < best_cost:
                    best_solution = current_solution
                    best_cost = current_cost
            
            temperature *= cooling_rate
        
        return best_solution
    
    best_solution = generate_initial_solution(numbers)
    best_cost = evaluate_solution(best_solution, numbers)
    
    for _ in range(max_iterations):
        solution = local_search(best_solution, numbers)
        solution = perturb(solution)
        solution = simulated_annealing(solution, numbers, initial_temp, cooling_rate, min_temp)
        cost = evaluate_solution(solution, numbers)
        if cost < best_cost:
            best_solution = solution
            best_cost = cost
    
    return best_solution, best_cost

# Exemplo de uso com leitura do arquivo
file_path = 'dados_literatura_Particao_Numeros.txt'  # Ajustar o caminho conforme necessário
instances = read_instances(file_path)

# Aplicando o algoritmo de GRASP com Simulated Annealing em cada instância
for idx, numbers in enumerate(instances):
    print(f"Instância {idx+1} ({len(numbers)} números):")
    best_solution, best_cost = grasp_with_simulated_annealing(numbers)
    print(f"Melhor solução: {best_solution}")
    print(f"Diferença mínima entre os subconjuntos: {best_cost}\n")

Instância 1 (15 números):
Melhor solução: [0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0]
Diferença mínima entre os subconjuntos: 1257426

Instância 2 (15 números):
Melhor solução: [0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1]
Diferença mínima entre os subconjuntos: 6137083

Instância 3 (15 números):
Melhor solução: [0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0]
Diferença mínima entre os subconjuntos: 4921863

Instância 4 (15 números):
Melhor solução: [1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0]
Diferença mínima entre os subconjuntos: 21246988

Instância 5 (15 números):
Melhor solução: [0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0]
Diferença mínima entre os subconjuntos: 9413138

Instância 6 (35 números):
Melhor solução: [1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1]
Diferença mínima entre os subconjuntos: 1340050

Instância 7 (35 números):
Melhor solução: [0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0,

In [59]:
def compare_results_with_parameters(instances, ils_pr_params, sa_params, kk_params, grasp_pr_params, grasp_sa_params, ils_sa_params):
    results_ils_pr = []
    results_sa = []
    results_kk = []
    results_grasp_pr = []
    results_grasp_sa = []
    results_ils_sa = []

    # Coletando resultados de ILS com Path Relinking
    for idx, numbers in enumerate(instances):
        best_solution, best_cost = ils_with_path_relinking(numbers, **ils_pr_params)
        results_ils_pr.append((f"Instância {idx+1}", best_cost))

    # Coletando resultados de Simulated Annealing
    for idx, numbers in enumerate(instances):
        best_solution, best_cost = simulated_annealing(numbers, **sa_params)
        results_sa.append((f"Instância {idx+1}", best_cost))

    # Coletando resultados de Karmarkar-Karp
    for idx, numbers in enumerate(instances):
        min_difference = karmarkar_karp(numbers)
        results_kk.append((f"Instância {idx+1}", min_difference))

    # Coletando resultados de GRASP com Path Relinking
    for idx, numbers in enumerate(instances):
        best_solution, best_cost = grasp_with_path_relinking(numbers, **grasp_pr_params)
        results_grasp_pr.append((f"Instância {idx+1}", best_cost))

    # Coletando resultados de GRASP com Simulated Annealing
    for idx, numbers in enumerate(instances):
        best_solution, best_cost = grasp_with_simulated_annealing(numbers, **grasp_sa_params)
        results_grasp_sa.append((f"Instância {idx+1}", best_cost))

    # Coletando resultados de ILS com Simulated Annealing
    for idx, numbers in enumerate(instances):
        best_solution, best_cost = ils_with_simulated_annealing(numbers, **ils_sa_params)
        results_ils_sa.append((f"Instância {idx+1}", best_cost))

    # Comparando os resultados e contando as vitórias
    compare_and_count_wins(results_ils_pr, results_sa, results_kk, results_grasp_pr, results_grasp_sa, results_ils_sa)

# Função para comparar os resultados e contar as vitórias
def compare_and_count_wins(results_ils_pr, results_sa, results_kk, results_grasp_pr, results_grasp_sa, results_ils_sa):
    wins = {
        'ILS_PR': 0,
        'SA': 0,
        'KK': 0,
        'GRASP_PR': 0,
        'GRASP_SA': 0,
        'ILS_SA': 0
    }
    
    print(f"{'Instância':<10} {'ILS_PR':<15} {'SA':<15} {'KK':<15} {'GRASP_PR':<15} {'GRASP_SA':<15} {'ILS_SA':<15}")
    print("="*90)
    for i in range(len(results_ils_pr)):
        inst = f"Instância {i+1}"
        ils_pr = results_ils_pr[i][1]
        sa = results_sa[i][1]
        kk = results_kk[i][1]
        grasp_pr = results_grasp_pr[i][1]
        grasp_sa = results_grasp_sa[i][1]
        ils_sa = results_ils_sa[i][1]
        
        # Encontrar o menor valor
        min_cost = min(ils_pr, sa, kk, grasp_pr, grasp_sa, ils_sa)
        
        # Contar qual algoritmo venceu
        if min_cost == ils_pr:
            wins['ILS_PR'] += 1
        elif min_cost == sa:
            wins['SA'] += 1
        elif min_cost == kk:
            wins['KK'] += 1
        elif min_cost == grasp_pr:
            wins['GRASP_PR'] += 1
        elif min_cost == grasp_sa:
            wins['GRASP_SA'] += 1
        elif min_cost == ils_sa:
            wins['ILS_SA'] += 1
        
        print(f"{inst:<10} {ils_pr:<15} {sa:<15} {kk:<15} {grasp_pr:<15} {grasp_sa:<15} {ils_sa:<15}")
    
    print("\nResultados:")
    for algo, count in wins.items():
        print(f"{algo}: {count} instâncias vencidas")

# Exemplo de uso com parâmetros ajustáveis
ils_pr_params = {
    'max_iterations': 100,
    'elite_size': 5
}

sa_params = {
    'initial_temp': 1000,
    'cooling_rate': 0.95,
    'min_temp': 1
}

grasp_pr_params = {
    'max_iterations': 100,
    'rcl_size': 5,
    'elite_size': 5
}

grasp_sa_params = {
    'max_iterations': 100,
    'rcl_size': 5,
    'initial_temp': 1000,
    'cooling_rate': 0.95,
    'min_temp': 1
}

ils_sa_params = {
    'max_iterations': 100,
    'initial_temp': 1000,
    'cooling_rate': 0.95,
    'min_temp': 1
}

# Comparando os resultados com parâmetros ajustáveis
compare_results_with_parameters(instances, ils_pr_params, sa_params, {}, grasp_pr_params, grasp_sa_params, ils_sa_params)

Instância  ILS_PR          SA              KK              GRASP_PR        GRASP_SA        ILS_SA         
Instância 1 16176584        1467432438      2043812         3584688         1257426         8048390        
Instância 2 63741595        908108941       49752055        6137083         3142889         6137083        
Instância 3 329587283       15981737        14440579        350627          49741           12293197       
Instância 4 758098          487695934       244401558       3637766         7370130         22070360       
Instância 5 74105806        36172           11307616        1257778         1257778         26624494       
Instância 6 8826416         380813308       24124           287052          534310          5083392        
Instância 7 3668200         89743128        537372          2176428         707344          1623392        
Instância 8 49580           564273208       195668          1773480         1057064         35790          
Instância 9 1441302         3

In [ ]:
import itertools

# Definindo listas de valores para cada parâmetro
max_iterations_list = [50, 100, 200]
elite_size_list = [3, 5, 7]
initial_temp_list = [500, 1000, 1500]
cooling_rate_list = [0.9, 0.95, 0.99]
min_temp_list = [1, 10, 50]
rcl_size_list = [3, 5, 7]

# Lista de seeds para garantir reprodutibilidade
seeds = [random.randint(0, 10000) for _ in range(50)]

# Função para executar os testes com diferentes combinações de parâmetros e seeds
def run_tests(instances):
    results = []

    for seed in seeds:
        random.seed(seed)
        for max_iterations, elite_size, initial_temp, cooling_rate, min_temp, rcl_size in itertools.product(
            max_iterations_list, elite_size_list, initial_temp_list, cooling_rate_list, min_temp_list, rcl_size_list
        ):
            ils_pr_params = {
                'max_iterations': max_iterations,
                'elite_size': elite_size
            }

            sa_params = {
                'initial_temp': initial_temp,
                'cooling_rate': cooling_rate,
                'min_temp': min_temp
            }

            grasp_pr_params = {
                'max_iterations': max_iterations,
                'rcl_size': rcl_size,
                'elite_size': elite_size
            }

            grasp_sa_params = {
                'max_iterations': max_iterations,
                'rcl_size': rcl_size,
                'initial_temp': initial_temp,
                'cooling_rate': cooling_rate,
                'min_temp': min_temp
            }

            ils_sa_params = {
                'max_iterations': max_iterations,
                'initial_temp': initial_temp,
                'cooling_rate': cooling_rate,
                'min_temp': min_temp
            }

            # Comparando os resultados com parâmetros ajustáveis
            compare_results_with_parameters(instances, ils_pr_params, sa_params, {}, grasp_pr_params, grasp_sa_params, ils_sa_params)

            # Armazenando os resultados
            results.append({
                'seed': seed,
                'max_iterations': max_iterations,
                'elite_size': elite_size,
                'initial_temp': initial_temp,
                'cooling_rate': cooling_rate,
                'min_temp': min_temp,
                'rcl_size': rcl_size
            })

    return results

# Executando os testes
results = run_tests(instances)

# Exibindo os resultados
for result in results:
    print(result)